In [ ]:
###############################################################################################################
### Author: Phinehas Lampman
### 
### Email: plampman@uidaho.edu
### 
### Purpose: Algorithm for generating spread direction lines between two fire front observations
###############################################################################################################

import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import LineString
import itertools


def calculate_azimuths(geometry_array):
    """
    Helper function for process_spread_lines() function below
    
    Extract coordinates from geometries and calculate azimuth for both line directions (trailing to leading and leading to trailing).
    
    Parameters:
    geometry_array: Array of LineString geometries passed from process_spread_lines() function below
    
    Returns:
    Dictionary containing all azimuth calculations and coordinate data for both directions (trailing and leading)
    """
    # Extract coordinates
    start_coords = np.array([line.coords[0] for line in geometry_array])
    end_coords = np.array([line.coords[-1] for line in geometry_array])
    
    ax, ay = start_coords[:, 0], start_coords[:, 1]
    bx, by = end_coords[:, 0], end_coords[:, 1]
    
    # Calculate raw direction angles for both directions
    degrees_trailing = np.rad2deg(np.arctan2(by - ay, bx - ax))
    degrees_leading = np.rad2deg(np.arctan2(ay - by, ax - bx))
    
    # Convert to standard azimuth (0-360 degrees) for both directions
    conditions = [
        (degrees_trailing < 90),
        (degrees_trailing > 90),
        ((degrees_trailing > 0) & (degrees_trailing <= 90)),
        (degrees_trailing == 0)
    ]
    
    choice = [
        (90 - degrees_trailing),
        (360 - degrees_trailing + 90),
        (90 - degrees_trailing),
        0
    ]
    
    spread_azimuth_trailing = np.select(conditions, choice, np.nan)
    
    conditions = [
        (degrees_leading < 90),
        (degrees_leading > 90),
        ((degrees_leading > 0) & (degrees_leading <= 90)),
        (degrees_leading == 0)
    ]
    
    choice = [
        (90 - degrees_leading),
        (360 - degrees_leading + 90),
        (90 - degrees_leading),
        0
    ]
    
    spread_azimuth_leading = np.select(conditions, choice, np.nan)
    
    # Calculate normalized azimuth values (-180 to 180) for both directions
    spread_az180_trailing = np.where(spread_azimuth_trailing > 180, spread_azimuth_trailing - 360, spread_azimuth_trailing)
    spread_az180_leading = np.where(spread_azimuth_leading > 180, spread_azimuth_leading - 360, spread_azimuth_leading)
    
    result = {
        'start_coords': start_coords,
        'end_coords': end_coords,
        'ax': ax, 
        'ay': ay,
        'bx': bx, 
        'by': by,
        'spread_azimuth_trailing': spread_azimuth_trailing,
        'spread_az180_trailing': spread_az180_trailing,
        'spread_azimuth_leading': spread_azimuth_leading,
        'spread_az180_leading': spread_az180_leading
    }
    
    return result

def process_spread_lines(points_df1, points_df2, front1_line, front2_line, output_path=None):
    """
    Create spread lines between two sets of points (front1 and front2 points) in both directions.
    
    Parameters:
    points_df1: First GeoDataFrame with point geometries (must have 'angle' column)
    points_df2: Second GeoDataFrame with point geometries (must have 'angle' column)
    QGIS "points along geometry" tool outputs an "angle" column in the correct manner
    -> angle (azimuth) of fire front line where the point is
    
    front1_line: LineString or MultiLineString representing the front1 (trailing front) line
    front2_line: LineString or MultiLineString representing the front2 (leading front) line
    output_path: Where to save the resulting combined GeoDataFrame
    
    Returns:
    Dictionary containing two GeoDataFrames: 'trailing' and 'leading' spread lines

    Output: if output_path is provided it will save a file with trailing and leading spread lines (concatenated)
    """
    
    # Verify required inputs exist
    if 'angle' not in points_df1.columns:
        raise ValueError("'angle' column not found in points_df1")
    if 'angle2' not in points_df2.columns:
        raise ValueError("'angle2' column not found in points_df2")
    if front1_line is None:
        raise ValueError("front1_line is required but was not provided")
    if front2_line is None:
        raise ValueError("front2_line is required but was not provided")
    
    # Unify geometries if they're collections
    from shapely.ops import unary_union
    if hasattr(front1_line, '__iter__') and not hasattr(front1_line, 'geom_type'):
        front1_line = unary_union(list(front1_line))
    if hasattr(front2_line, '__iter__') and not hasattr(front2_line, 'geom_type'):
        front2_line = unary_union(list(front2_line))
    
    results = {}
    
    # Process trailing direction (front1 to front2)
    print('Processing trailing front to leading front spread lines...')
    
    # Get point geometries
    geom1 = points_df1.geometry.values
    geom2 = points_df2.geometry.values
    
    # Create lines between all combinations of points (direction from points1 to points2)
    geom = [LineString([p1, p2]) for p1, p2 in itertools.product(geom1, geom2)]
    
    # Create GeoDataFrame
    trailing_result = gpd.GeoDataFrame({'geometry': geom}, crs="EPSG:26914")
    trailing_result['length_m'] = trailing_result.length
    trailing_result['direction'] = "trailing"  # Add direction identifier
    print(f'Created {len(trailing_result)} trailing lines')
    
    # Prepare front1 join dataframe
    front1_join = points_df1.copy()
    front1_join['xy_1'] = [tuple(p.coords[0]) for p in front1_join.geometry]
    front1_join['Pt_ID1'] = np.arange(len(front1_join))
    front1_join = front1_join[['xy_1', 'Pt_ID1', 'angle']]
    
    # Prepare front2 join dataframe
    front2_join = points_df2.copy()
    front2_join['xy_2'] = [tuple(p.coords[0]) for p in front2_join.geometry]
    front2_join['Pt_ID2'] = np.arange(len(front2_join))
    front2_join = front2_join[['xy_2', 'Pt_ID2', 'angle2']]
    
    # Calculate azimuths for trailing direction
    azimuth_data = calculate_azimuths(trailing_result.geometry.values)
    
    # Add azimuth data to trailing result
    trailing_result['spreadAZ'] = azimuth_data['spread_azimuth_trailing']
    trailing_result['spreadAz180'] = azimuth_data['spread_az180_trailing']
    
    # Prepare for joining
    trailing_result['xy1'] = [tuple(coord) for coord in azimuth_data['start_coords']]
    trailing_result['xy2'] = [tuple(coord) for coord in azimuth_data['end_coords']]
    
    # Join data for trailing
    print('Joining trailing with fire front data...')
    trailing_result = trailing_result.merge(front1_join, how='left', left_on='xy1', right_on='xy_1')
    trailing_result = trailing_result.merge(front2_join, how='left', left_on='xy2', right_on='xy_2')
    
    # Calculate normalized angle values (-180 to 180)
    trailing_result['Front1Az180'] = np.where(trailing_result['angle'] > 180, 
                                     trailing_result['angle'] - 360, 
                                     trailing_result['angle'])
    trailing_result['Front2Az180'] = np.where(trailing_result['angle2'] > 180, 
                                     trailing_result['angle2'] - 360, 
                                     trailing_result['angle2'])
    
    # Calculate angle differences for trailing
    trailing_result['angle_diff1'] = np.abs(trailing_result['spreadAz180'] - trailing_result['Front1Az180'])
    trailing_result['angle_diff2'] = np.abs(trailing_result['spreadAz180'] - trailing_result['Front2Az180'])
    
    # Apply angle filtering for trailing
    print("Filtering trailing lines by angle criteria...")
    before_count = len(trailing_result)
    trailing_result = trailing_result[(trailing_result.angle_diff1 > 60) & (trailing_result.angle_diff1 < 120)]
    trailing_result = trailing_result[(trailing_result.angle_diff2 > 60) & (trailing_result.angle_diff2 < 120)]
    after_count = len(trailing_result)
    print(f"Kept {after_count} of {before_count} lines after angle filtering")
    
    # Filter out trailing lines that intersect with fire front lines
    print("Filtering trailing lines that intersect with fire fronts...")
    valid_indices = []
    
    # Create a combined front line for faster intersection checking
    from shapely.ops import unary_union
    if hasattr(front1_line, '__iter__') and not hasattr(front1_line, 'geom_type'):
        front1_line = unary_union(list(front1_line))
    if hasattr(front2_line, '__iter__') and not hasattr(front2_line, 'geom_type'):
        front2_line = unary_union(list(front2_line))
    combined_fronts = unary_union([front1_line, front2_line])
    
    # Process trailing in batches to avoid memory issues with large datasets
    batch_size = 100000
    total_rows = len(trailing_result)
    
    for i in range(0, total_rows, batch_size):
        batch_end = min(i + batch_size, total_rows)
        print(f"Processing trailing batch {i}-{batch_end} of {total_rows}...")
        
        # Check intersections with combined fronts
        for idx, line in enumerate(trailing_result.iloc[i:batch_end].geometry):
            # We want a proper crossing, not just touching at endpoints
            start_point = line.coords[0]
            end_point = line.coords[-1]
                
            # Create buffered endpoints
            from shapely.geometry import Point
            start_buffer = Point(start_point).buffer(0.5)
            end_buffer = Point(end_point).buffer(0.5)
            
            # Check if line crosses front lines (excluding endpoints)
            intersection = line.intersection(combined_fronts)
            if intersection.is_empty or (
                intersection.geom_type in ['Point', 'MultiPoint'] and 
                (start_buffer.contains(intersection) or end_buffer.contains(intersection))
            ):
                valid_indices.append(i + idx)
    
    # Apply the filter for trailing
    if valid_indices:
        trailing_result = trailing_result.iloc[valid_indices]
        print(f"Kept {len(trailing_result)} non-intersecting trailing lines after filtering")
    else:
        print("Warning: All trailing lines intersect with fire fronts, no valid lines remain")
        # Use empty DataFrame for trailing to avoid errors in subsequent steps
        trailing_result = gpd.GeoDataFrame(columns=trailing_result.columns, geometry='geometry', crs=trailing_result.crs)
    
    # Verify we still have data for all front2 points in trailing
    covered_pts = set(trailing_result['Pt_ID2'].unique())
    all_pts = set(range(len(points_df2)))
    missing_pts = all_pts - covered_pts
    if missing_pts:
        print(f"Warning: {len(missing_pts)} front2 points have no valid trailing lines after filtering")
    
    # Get shortest line for each point in front2 (Pt_ID2) for trailing
    trailing_result = trailing_result.loc[trailing_result.groupby('Pt_ID2')['length_m'].nsmallest(1).index.get_level_values(1)]
    
    # Clean up trailing result
    trailing_result = trailing_result.drop(['xy1', 'xy2', 'xy_1', 'xy_2'], axis=1)
    
    # Store trailing result
    results['trailing'] = trailing_result
    
    # Process leading direction
    print('Processing leading to trailing spread lines...')
    
    # Create lines between all combinations of points (direction from points2 to points1)
    geom = [LineString([p2, p1]) for p1, p2 in itertools.product(geom1, geom2)]
    
    # Create GeoDataFrame
    leading_result = gpd.GeoDataFrame({'geometry': geom}, crs="EPSG:26914")
    leading_result['length_m'] = leading_result.length
    leading_result['direction'] = "leading"  # Add direction identifier
    print(f'Created {len(leading_result)} leading lines')
    
    # Calculate azimuths for leading direction
    azimuth_data = calculate_azimuths(leading_result.geometry.values)
    
    # Add azimuth data to leading result
    leading_result['spreadAZ'] = azimuth_data['spread_azimuth_leading']
    leading_result['spreadAz180'] = azimuth_data['spread_az180_leading']
    
    # Prepare for joining
    leading_result['xy1'] = [tuple(coord) for coord in azimuth_data['start_coords']]
    leading_result['xy2'] = [tuple(coord) for coord in azimuth_data['end_coords']]
    
    # Join data for leading
    print('Joining leading with fire front data...')
    leading_result = leading_result.merge(front2_join, how='left', left_on='xy1', right_on='xy_2')
    leading_result = leading_result.merge(front1_join, how='left', left_on='xy2', right_on='xy_1')
    
    # Calculate normalized angle values (-180 to 180)
    leading_result['Front1Az180'] = np.where(leading_result['angle'] > 180, 
                                    leading_result['angle'] - 360, 
                                    leading_result['angle'])
    leading_result['Front2Az180'] = np.where(leading_result['angle2'] > 180, 
                                    leading_result['angle2'] - 360, 
                                    leading_result['angle2'])
    
    # Calculate angle differences for leading
    leading_result['angle_diff1'] = np.abs(leading_result['spreadAz180'] - leading_result['Front2Az180'])
    leading_result['angle_diff2'] = np.abs(leading_result['spreadAz180'] - leading_result['Front1Az180'])
    
    # Apply angle filtering for leading
    print("Filtering leading lines by angle criteria...")
    before_count = len(leading_result)
    leading_result = leading_result[(leading_result.angle_diff1 > 60) & (leading_result.angle_diff1 < 120)]
    leading_result = leading_result[(leading_result.angle_diff2 > 60) & (leading_result.angle_diff2 < 120)]
    after_count = len(leading_result)
    print(f"Kept {after_count} of {before_count} lines after angle filtering")
    
    # Filter out leading lines that intersect with fire front lines
    print("Filtering leading lines that intersect with fire fronts...")
    valid_indices = []
    
    # Process leading in batches to avoid memory issues with large datasets
    batch_size = 100000
    total_rows = len(leading_result)
    
    for i in range(0, total_rows, batch_size):
        batch_end = min(i + batch_size, total_rows)
        print(f"Processing leading batch {i}-{batch_end} of {total_rows}...")
        
        # Check intersections with combined fronts
        for idx, line in enumerate(leading_result.iloc[i:batch_end].geometry):
            # We want a proper crossing, not just touching at endpoints
            start_point = line.coords[0]
            end_point = line.coords[-1]
                
            # Create buffered endpoints
            from shapely.geometry import Point
            start_buffer = Point(start_point).buffer(0.5)
            end_buffer = Point(end_point).buffer(0.5)
            
            # Check if line crosses front lines (excluding endpoints)
            intersection = line.intersection(combined_fronts)
            if intersection.is_empty or (
                intersection.geom_type in ['Point', 'MultiPoint'] and 
                (start_buffer.contains(intersection) or end_buffer.contains(intersection))
            ):
                valid_indices.append(i + idx)
    
    # Apply the filter for leading
    if valid_indices:
        leading_result = leading_result.iloc[valid_indices]
        print(f"Kept {len(leading_result)} non-intersecting leading lines after filtering")
    else:
        print("Warning: All leading lines intersect with fire fronts, no valid lines remain")
        leading_result = gpd.GeoDataFrame(columns=leading_result.columns, geometry='geometry', crs=leading_result.crs)
    
    # Verify we still have data for all front1 points in leading
    covered_pts = set(leading_result['Pt_ID1'].unique())
    all_pts = set(range(len(points_df1)))
    missing_pts = all_pts - covered_pts
    if missing_pts:
        print(f"Warning: {len(missing_pts)} front1 points have no valid leading lines after filtering")
    
    # Get shortest line for each point in front1 (Pt_ID1) for leading
    leading_result = leading_result.loc[leading_result.groupby('Pt_ID1')['length_m'].nsmallest(1).index.get_level_values(1)]
    
    # Clean up leading result
    leading_result = leading_result.drop(['xy1', 'xy2', 'xy_1', 'xy_2'], axis=1)
    
    # Store leading result
    results['leading'] = leading_result
    
    # Combine results for output
    if output_path:
        combined_results = pd.concat([results['trailing'], results['leading']], ignore_index=True)
        print(f'Combined {len(combined_results)} total spread lines')
        print(f'Writing combined results to {output_path}')
        combined_results.to_file(output_path)
    
    # Report counts
    print(f'Processed {len(results["trailing"])} trailing lines and {len(results["leading"])} leading lines')
    
    return results


# Load your points data
front1_points = gpd.read_file('./Insert Points1') # Trailing front points 
print("Loaded Front1 points")

front2_points = gpd.read_file('./Insert Points2') # Leading front points 
front2_points = front2_points.rename(columns={"angle": "angle2"}) # angle changed to angle2 for leading points
print("Loaded Front2 points")

# Load fire front polylines - these are required
front1_lines = gpd.read_file('./Insert Trailing Front Polylines1') # Trailing front polylines 
front1_line = front1_lines.geometry.unary_union
print("Loaded Front1 line geometry")

front2_lines = gpd.read_file('./Insert Trailing Front Polylines2') # Leading front polylines
front2_line = front2_lines.geometry.unary_union
print("Loaded Front2 line geometry")

# Process in both directions and get results
output_path = './FrontX_to_FrontX_spreadlines.gpkg' # Output file

# Run algorithm
results = process_spread_lines(
    front1_points, 
    front2_points, 
    front1_line, 
    front2_line, 
    output_path=output_path
)

trailing_lines = results['trailing']
leading_lines = results['leading']